In [1]:
import nibabel as nib
import numpy as np
import glob
import pandas as pd
from torchvision import transforms
import torch
from PIL import Image
import random
import pickle

In [ ]:
df = pd.read_csv("/space/mcdonald-syn01/1/projects/jsawant/DSC250/data_csvs/train.csv")
df = df.loc[df["HC_vs_LTLE_vs_RTLE_string"].isin(["right","left","HC"])]
hc_df = df[df["HC_vs_LTLE_vs_RTLE_string"]=="HC"]
tle_df = df[df["HC_vs_LTLE_vs_RTLE_string"]!="HC"]

In [3]:
def preprocess(image):
    image = image[:,1:137,:]
    base_transform = transforms.Compose([
            transforms.Resize((112, 112)),  # First resize to 113x113
            #transforms.Pad(padding=(8, 8, 8, 8), fill=0),  # Add asymmetric padding to reach 128x128
            transforms.ToTensor(),
            #transforms.Lambda(lambda x: x.mul(2).sub(1))
        ])
    slices = []
    for i in range(image.shape[1]):
        slice_img = Image.fromarray(image[:, i, :])
        # Apply base transforms
        slice_tensor = base_transform(slice_img)
        slices.append(slice_tensor)
        
    image = torch.stack(slices, dim=0)  # Shape: [115, 1, 128, 128]
    image = image.squeeze(1)
    image = image.permute(1,0,2)

    return image

In [4]:
def save_nii(path, idx):
    if ".nii" in path:
        img = nib.load(path)
        img = img.get_fdata()
        img = (img - np.mean(img)) / np.std(img) 
        img = preprocess(img)
        img = img.cpu().numpy()
    else:
        with open(path,"rb") as f:
            synth = pickle.load(f)
        img = synth["image"]
    affine = np.eye(4)
    img = nib.Nifti1Image(img, affine)
    nib.save(img, f"/space/mcdonald-syn01/1/projects/jsawant/DSC250/nii_test/{idx}.nii")

In [5]:
final_df = pd.DataFrame(columns=["ID", "filepath", "label", "Real/synthetic"])


In [6]:
hc_df_f = hc_df.sample(12)
tle_df_f = tle_df.sample(12)
real_df = pd.concat([hc_df_f, tle_df_f])
real_df

,sbj_nme,file,HC_vs_EPD_numeric,LTLE_vs_RTLE_numeric,HC_vs_LTLE_vs_RTLE_numeric,HC_vs_EPD_MRI_Plus_numeric,LTLE_vs_RTLE_MRI_Plus_numeric,HC_vs_LTLE_vs_RTLE_MRI_Plus_numeric,HC_vs_EPD_string,LTLE_vs_RTLE_string,HC_vs_LTLE_vs_RTLE_string,HC_vs_EPD_MRI_Plus_string,LTLE_vs_RTLE_MRI_Plus_string,HC_vs_LTLE_vs_RTLE_MRI_Plus_string
2570,sub-UNAM04202,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
385,sub-EmoryEEC037,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
678,sub-EmoryEMUC0004,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
652,sub-PITC0060,/space/mcdonald-syn01/1/BIDS//CAPES//derivativ...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
1303,sub-ECPEC2076,/space/mcdonald-syn01/1/BIDS//ECP_dataset_bids...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
238,sub-LangoneNC0156,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
99,sub-EMOC0012,/space/mcdonald-syn01/1/BIDS//CAPES//derivativ...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
2551,sub-EMOC0007,/space/mcdonald-syn01/1/BIDS//CAPES//derivativ...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
1047,sub-20130127103109,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC
2071,sub-EPICZCON008,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,1.0,NaN,1.0,1.0,NaN,1.0,HC,NaN,HC,HC,NaN,HC


In [7]:
indexes = list(np.arange(1, 49, 1))
random.shuffle(indexes)
# Save NIfTI files for the sampled real data
for i in range(len(real_df)):
    path = real_df["file"].iloc[i]
    label = real_df["HC_vs_LTLE_vs_RTLE_string"].iloc[i]
    label = "TLE" if label in ['right','left'] else "HC"
    idx = indexes.pop()
    row_data = (idx,path,label,"Real")
    final_df.loc[len(final_df)] = row_data
    save_nii(path, idx)

In [8]:
synth_hc_files = glob.glob("/space/mcdonald-syn01/1/projects/jsawant/DSC250/synthetic_data_pkls_HC_gan/*.pkl")
synth_tle_files = glob.glob("/space/mcdonald-syn01/1/projects/jsawant/DSC250/synthetic_data_pkls_TLE_gan/*.pkl")
synth_final_files = random.choices(synth_hc_files, k=12)
synth_final_files.extend(random.choices(synth_tle_files, k=12))

In [9]:
for path in synth_final_files:
    with open(path,"rb") as f:
        synth = pickle.load(f)
    #img = synth["image"]
    label = "HC" if synth['label']==0 else "TLE"
    idx = indexes.pop()
    row_data = (idx,path,label,"Synthetic")
    final_df.loc[len(final_df)] = row_data
    save_nii(path, idx)

In [10]:
final_df.to_csv("/space/mcdonald-syn01/1/projects/jsawant/DSC250/qualitative_results.csv")

In [11]:
indexes

[]

In [12]:
final_df

,ID,filepath,label,Real/synthetic
0,7,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,HC,Real
1,37,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,HC,Real
2,14,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,HC,Real
3,40,/space/mcdonald-syn01/1/BIDS//CAPES//derivativ...,HC,Real
4,4,/space/mcdonald-syn01/1/BIDS//ECP_dataset_bids...,HC,Real
5,43,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,HC,Real
6,9,/space/mcdonald-syn01/1/BIDS//CAPES//derivativ...,HC,Real
7,1,/space/mcdonald-syn01/1/BIDS//CAPES//derivativ...,HC,Real
8,20,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,HC,Real
9,5,/space/mcdonald-syn01/1/BIDS//enigma_conglom//...,HC,Real
